In [50]:

import pandas as pd
import os
from datetime import datetime
import pytz
from sklearn.impute import KNNImputer

In [29]:
# 读取因子数据
factors_df = pd.read_csv('/Users/ryan/Desktop/project/collection_code/factor_data/ff_factors.csv', parse_dates=['Date'])


In [40]:
folder_path = '/Users/ryan/Desktop/project/collection_code/stock_data'
# 获取文件夹中所有 CSV 文件的路径
csv_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

# 读取所有 CSV 文件并合并到一个 DataFrame 中，并添加文件名作为新的一列
stock_returns_df = pd.concat([pd.read_csv(file, parse_dates=['Date']).assign(FileName=os.path.basename(file)) for file in csv_files], ignore_index=True)

columns = stock_returns_df.columns.tolist()
# 将 FileName 列移动到第一列
columns = ['FileName'] + [col for col in columns if col != 'FileName']

# 重新排列数据框的列顺序
stock_returns_df = stock_returns_df[columns]

/var/folders/cw/yxxjm9s13cz54l3wm4btmtnc0000gn/T/ipykernel_36548/4088704385.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  stock_returns_df = pd.concat([pd.read_csv(file, parse_dates=['Date']).assign(FileName=os.path.basename(file)) for file in csv_files], ignore_index=True)


In [41]:
# 确保 stock_returns_df 中的 Date 列是 datetime64[ns] 类型
stock_returns_df['Date'] = pd.to_datetime(stock_returns_df['Date'], utc=True).dt.tz_localize(None)

# 确保 factors_df 中的 Date 列是 datetime64[ns] 类型
factors_df['Date'] = pd.to_datetime(factors_df['Date'])

# 修改日期格式，去掉时间部分和时区信息
def format_date(date):
    # 确保日期是 datetime 类型
    if not isinstance(date, pd.Timestamp):
        date = pd.to_datetime(date)
    # 格式化日期为 'YYYY-MM-DD' 格式
    return date.strftime('%Y-%m-%d')

# 应用日期
stock_returns_df['Date'] = stock_returns_df['Date'].apply(format_date)
factors_df['Date'] = factors_df['Date'].apply(format_date)

In [58]:
# 合并数据
merged_df = pd.merge(stock_returns_df, factors_df, on='Date', how='inner')

output_file = 'merged_data.csv'
merged_df.to_csv(output_file, index=False)

In [56]:
#数据总览
print(merged_df.describe()) 
print(merged_df['FileName'].value_counts()) 

                Open           High            Low          Close  \
count  275364.000000  275364.000000  275364.000000  275364.000000   
mean      164.710327     166.723615     162.710308     164.761656   
std       315.537898     319.385144     311.966732     315.741006   
min         3.475000       3.562000       3.411000       3.483000   
25%        55.001116      55.655305      54.301407      54.999674   
50%        99.899141     101.203788      98.639906      99.944542   
75%       189.887805     192.167894     187.489015     189.926517   
max      7793.580078    7840.000000    7721.470215    7791.540039   

           Dividends   Stock Splits    momentum_5d   momentum_10d  \
count  275364.000000  275364.000000  275364.000000  275364.000000   
mean        0.008791       0.000401       0.001713       0.003392   
std         0.131118       0.069810       0.045768       0.063952   
min         0.000000       0.000000      -0.531282      -0.605622   
25%         0.000000       0.0000

In [ ]:
# Momentum 和 RSI 有缺失值（类似移动平均方法导致的树数值缺失），这里尝试了使用 KNN 算法填充缺失值
columns_to_impute=['momentum_5d','momentum_10d','momentum_20d','RSI']
df_to_impute = merged_df[columns_to_impute]

imputer = KNNImputer(n_neighbors=5)
df_imputed = imputer.fit_transform(df_to_impute)

merged_df[columns_to_impute]=df_imputed

print(merged_df)

             FileName        Date       Open       High        Low      Close  \
0        WMT_full.csv  2022-01-03  45.995855  46.248193  45.679632  46.203476   
1        WMT_full.csv  2022-01-04  46.008645  46.360004  45.280379  45.357037   
2        WMT_full.csv  2022-01-05  45.628539  46.618727  45.593403  45.970310   
3        WMT_full.csv  2022-01-06  45.874486  46.200290  45.484799  45.842545   
4        WMT_full.csv  2022-01-07  45.836152  46.430268  45.663668  46.280140   
...               ...         ...        ...        ...        ...        ...   
275359  MDLZ_full.csv  2024-03-13  70.156882  70.332129  69.524055  69.601936   
275360  MDLZ_full.csv  2024-03-14  69.514312  69.621407  68.628341  69.085930   
275361  MDLZ_full.csv  2024-03-15  69.124875  69.183288  68.287587  68.881477   
275362  MDLZ_full.csv  2024-03-18  69.124879  69.709031  68.949632  69.426689   
275363  MDLZ_full.csv  2024-03-19  69.767451  70.526850  69.670086  70.341866   

          Volume  Dividends

In [52]:
# 导出来的数据中，momentum_5d、momentum_10d、momentum_20d 和 RSI 列中的缺失值已经被填充了（但是需要检验
output_file = ' imputed_data.csv'
merged_df.to_csv(output_file, index=False)

In [ ]:
#归一化数据
from sklearn.preprocessing import MinMaxScaler, StandardScaler
# 选择需要归一化的数值列 也可以继续添加需要归一化的列
numeric_columns = [
    'Open', 'High', 'Low', 'Close', 'Volume', 'momentum_5d', 'momentum_10d', 'momentum_20d', 'RSI', 'MACD',
    'Signal_Line', 'MACD_Histogram', 'marketCap', 'priceToBook', 'trailingPE', 'forwardPE', 'profitMargins',
    'net_income_growth', 'Adj Close', 'Mkt-RF', 'SMB', 'HML', 'RF'
]
# 提取数值列
numeric_data = merged_df[numeric_columns]

# 应用StandardScaler进行归一化
scaler = StandardScaler()
data_normalized = scaler.fit_transform(numeric_data)

# 将归一化后的数值列放回原数据框
merged_df[numeric_columns] = data_normalized

# 查看归一化后的数据
print(merged_df)

/Users/ryan/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Users/ryan/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/Users/ryan/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


             FileName        Date      Open      High       Low     Close  \
0        WMT_full.csv  2022-01-03 -0.376230 -0.377211 -0.375139 -0.375493   
1        WMT_full.csv  2022-01-04 -0.376189 -0.376861 -0.376419 -0.378173   
2        WMT_full.csv  2022-01-05 -0.377394 -0.376051 -0.375415 -0.376231   
3        WMT_full.csv  2022-01-06 -0.376614 -0.377361 -0.375764 -0.376636   
4        WMT_full.csv  2022-01-07 -0.376736 -0.376641 -0.375190 -0.375250   
...               ...         ...       ...       ...       ...       ...   
275359  MDLZ_full.csv  2024-03-13 -0.299659 -0.301804 -0.298706 -0.301386   
275360  MDLZ_full.csv  2024-03-14 -0.301695 -0.304029 -0.301577 -0.303020   
275361  MDLZ_full.csv  2024-03-15 -0.302929 -0.305401 -0.302670 -0.303668   
275362  MDLZ_full.csv  2024-03-18 -0.302929 -0.303755 -0.300548 -0.301941   
275363  MDLZ_full.csv  2024-03-19 -0.300893 -0.301194 -0.298238 -0.299042   

          Volume  Dividends  Stock Splits   momentum_5d  ...  priceToBook  